In [20]:
"""
IMPROVING ON THE 50/150 MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 20 YEARS OF DATA

First,  we use the backtesting library to find a similar strategy to the conventional 50/150 strategy. We do this by 
using the backtesting library to test combinations of short SMAs between 40 and 60 with long SMAs between 130 and 170. 

Next, we confirm the results using our own backtesting and metrics.

The backtesting library choose short = 57, long =130 . It tested 800 combinations using short = 40 to 60 with a step of 1
and long = 130 to 170 with a step of 1. However, this did not beat a buy and hold strategy. 

A backtest showed a 135% return for the 57/130 strategy.

A backtest showed a 87% return using the 50/150 strategy. So we were able to improve upon the 50/150 strategy. 
 
However, both strategies underperformed the 215% return of a buy and hold strategy. Most of the underperformance
occurred at the coronavirus bear market.


"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

%matplotlib inline

In [21]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [22]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-07-02,314.2379,315.70,310.5962,312.19,312.19,69214995.0,0.0,1.0
2020-07-01,309.5700,311.85,309.0371,310.57,310.57,71910372.0,0.0,1.0
2020-06-30,303.9900,310.20,303.8200,308.36,308.36,112828251.0,0.0,1.0
2020-06-29,301.4100,304.61,298.9300,304.46,304.46,79411577.0,0.0,1.0
2020-06-26,306.1600,306.39,299.4200,300.05,300.05,127811745.0,0.0,1.0


In [23]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2000-07-03,145.4375,147.4375,145.1875,147.2812,100.6579,1436600.0,0.0,1.0
2000-07-05,146.3750,146.6562,144.3750,144.6250,98.8425,2748200.0,0.0,1.0
2000-07-06,144.9375,146.4687,144.2187,145.7500,99.6114,5963200.0,0.0,1.0
2000-07-07,146.6875,148.7812,146.2500,148.0937,101.2132,3034800.0,0.0,1.0
2000-07-10,147.8750,148.9062,147.5312,147.8437,101.0423,2816100.0,0.0,1.0


In [24]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.sort_index(ascending=True, inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2000-07-03,145.4375,147.4375,145.1875,100.6579,1436600.0
2000-07-05,146.3750,146.6562,144.3750,98.8425,2748200.0
2000-07-06,144.9375,146.4687,144.2187,99.6114,5963200.0
2000-07-07,146.6875,148.7812,146.2500,101.2132,3034800.0
2000-07-10,147.8750,148.9062,147.5312,101.0423,2816100.0


In [6]:
#Function to do a backtest using Backtesting library. This function will find the best SMA cross strategy by analyzing
#different combinations of long short smas.

#Change the ranges in the stats for different scenarios. For example, range(5,365,5) means to test smas 
#between 5 and 365 days and use a 5 day step. 

def backtest():
    class SmaCross(Strategy):
    
    #n1 =short, n2 = long, they can be anything at the start of the optimization
        n1=1
        n2=365
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=100000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(40, 60, 1),
                    n2=range(130, 170, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:33: UserWarning:

Searching best of 800 configurations.



Start                        2000-06-30 00:00:00
End                          2020-07-02 00:00:00
Duration                      7307 days 00:00:00
Exposure [%]                             93.4857
Equity Final [$]                          235372
Equity Peak [$]                           398306
Return [%]                               135.372
Buy & Hold Return [%]                    215.799
Max. Drawdown [%]                       -52.5541
Avg. Drawdown [%]                       -10.4175
Max. Drawdown Duration         974 days 00:00:00
Avg. Drawdown Duration          98 days 00:00:00
# Trades                                      33
Win Rate [%]                             54.5455
Best Trade [%]                            42.979
Worst Trade [%]                         -11.3787
Avg. Trade [%]                           4.74574
Max. Trade Duration           1097 days 00:00:00
Avg. Trade Duration            207 days 00:00:00
Expectancy [%]                           9.82757
SQN                 

In [25]:
# To confirm the above results we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2000-07-03,100.6579
2000-07-05,98.8425
2000-07-06,99.6114
2000-07-07,101.2132
2000-07-10,101.0423


In [29]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 57
long_window = 139

#Set names of the windows
short_name = "SMA57"
long_name = "SMA130"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.head(1000)


,Close,SMA57,SMA130,Signal,Buy/Sell
date,,,,,
2000-07-03,100.6579,NaN,NaN,0.0,NaN
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0
2000-07-10,101.0423,NaN,NaN,0.0,0.0
2000-07-11,101.2559,NaN,NaN,0.0,0.0
2000-07-12,101.9180,NaN,NaN,0.0,0.0
2000-07-13,102.3665,NaN,NaN,0.0,0.0
2000-07-14,103.3703,NaN,NaN,0.0,0.0


In [12]:
len(signals_df)

5033

In [14]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[[short_name, long_name]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [15]:
signals_df.head()

,Close,SMA57,SMA130,Signal,Buy/Sell
date,,,,,
2000-06-30,99.2910,NaN,NaN,0.0,NaN
2000-07-03,100.6579,NaN,NaN,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0


In [16]:

# Add share and cash columns

signals_df["Shares"] = float(0)
signals_df["Portfolio Cash"] = float(100000)

#Set up starting cash variable for sma strategy and buy and hold strategy

cash = 100000
buy_and_hold_cash = 100000


#Iterate through each row, use combinations of signal and buy/sell points to set shares and portfolio cash for each row.

for index, row in signals_df.iterrows():
    
    #If at a buy point, calculate the number of shares using total cash available and set the cash to zero
    
    if row['Buy/Sell'] == 1.0:
       
        shares = cash/row['Close']
        row['Shares'] = shares
        row['Portfolio Cash']= 0 
       
    #If at sell point, calculate the cash generated from the sale (shares X closing price) and set the shares to zero, then
    #update the cash variable with the new cash balance'''
    
    elif row['Buy/Sell'] == -1.0:
        row['Portfolio Cash']= shares*row['Close']
        cash = row['Portfolio Cash']
        row['Shares']= 0
        
    #If we are not at a buy or sell point, look to signal column, if the signal is buy, then set cash to zero and update shares.
    # If the signal is sell, update the cash column using cash variable
    else:
     
        row['Buy/Sell'] == 0
        if row['Signal']== 1:
            row['Portfolio Cash'] = 0
            row["Shares"]= shares
        elif row['Signal'] == 0:
            row['Portfolio Cash'] = cash
        
# Add a Portfolio Total column
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + (signals_df['Shares']*signals_df['Close'])

# Calculate how many shares a buy and hold investor could buy at the start of the comparison. Use end of long window as start. 
buy_and_hold_shares = buy_and_hold_cash/signals_df['Close'].iloc[long_window-1]

# Add a buy and hold total column for comparison, set the first 225 (long window) to 100000 so both portfolios are at 100000 at the
#end of the long window calculation
signals_df["Buy&Hold Total"]= buy_and_hold_shares * signals_df['Close']
signals_df["Buy&Hold Total"][0:long_window-1] = buy_and_hold_cash      
        
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()  
signals_df['Buy&Hold Daily Returns']= signals_df['Buy&Hold Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cum Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df['Buy&Hold Cum Returns'] = (1 + signals_df['Buy&Hold Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(50)

,Close,SMA57,SMA130,Signal,Buy/Sell,Shares,Portfolio Cash,Portfolio Total,Buy&Hold Total,Portfolio Daily Returns,Buy&Hold Daily Returns,Portfolio Cum Returns,Buy&Hold Cum Returns
date,,,,,,,,,,,,,
2000-06-30,99.2910,NaN,NaN,0.0,NaN,0.0,100000.0,100000.0,100000.0,NaN,NaN,NaN,NaN
2000-07-03,100.6579,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-10,101.0423,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-11,101.2559,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-12,101.9180,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-13,102.3665,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0


In [17]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [18]:
#Plot the SMA Strategy vs the Buy & Hold strategy

signals_df[["Buy&Hold Total", "Portfolio Total"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [28]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest', 'Buy&Hold']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [29]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'][0] = signals_df['Portfolio Cum Returns'][-1]
portfolio_evaluation_df.loc['Cumulative Returns'][1] = signals_df['Buy&Hold Cum Returns'][-1]

portfolio_evaluation_df



,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,1.96838,2.38418
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [30]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'][0] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return'][1] = (signals_df['Buy&Hold Daily Returns'].mean() * 252)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0622261,0.0799636
Cumulative Returns,1.96838,2.38418
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [31]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'][0] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility'][1] = (signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0622261,0.0799636
Cumulative Returns,1.96838,2.38418
Annual Volatility,0.124056,0.194362
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [32]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'][0] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df.loc['Sharpe Ratio'][1] = (
    signals_df['Buy&Hold Daily Returns'].mean() * 252) / (
    signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252)
)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0622261,0.0799636
Cumulative Returns,1.96838,2.38418
Annual Volatility,0.124056,0.194362
Sharpe Ratio,0.501597,0.411416
Sortino Ratio,NaN,NaN


In [33]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

sortino_ratio_df2 = signals_df[['Buy&Hold Daily Returns']].copy()
sortino_ratio_df2.loc[:,'Downside Returns1'] = 0

target = 0
mask = sortino_ratio_df2['Buy&Hold Daily Returns'] < target
sortino_ratio_df2.loc[mask, 'Downside Returns1'] = sortino_ratio_df2['Buy&Hold Daily Returns']**2


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0622261,0.0799636
Cumulative Returns,1.96838,2.38418
Annual Volatility,0.124056,0.194362
Sharpe Ratio,0.501597,0.411416
Sortino Ratio,NaN,NaN


In [34]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][0] = sortino_ratio

down_stdev = np.sqrt(sortino_ratio_df2['Downside Returns1'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df2['Buy&Hold Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][1] = sortino_ratio





portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0622261,0.0799636
Cumulative Returns,1.96838,2.38418
Annual Volatility,0.124056,0.194362
Sharpe Ratio,0.501597,0.411416
Sortino Ratio,0.685647,0.578024


In [35]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [36]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Buy/Sell'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Shares']*row['Close']
        share_size = row['Shares']
        entry_share_price = row['Close']

    elif row['Buy/Sell'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Total']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2002-01-17,2002-05-07,1263.324920,79.1562,73.3766,100000.000000,92698.487295,-7301.512705
1,SPY,2003-01-02,2003-02-24,1439.325870,64.4041,59.2628,92698.487295,85298.481197,-7400.006098
2,SPY,2003-05-20,2004-07-02,1299.380478,65.6455,81.7114,85298.481197,106174.198025,20875.716828
3,SPY,2004-10-28,2005-05-17,1290.134489,82.2970,86.4658,106174.198025,111552.510682,5378.312657
4,SPY,2005-07-08,2006-07-10,1245.358737,89.5746,95.3522,111552.510682,118747.695318,7195.184636
5,SPY,2006-09-15,2007-10-04,1191.901291,99.6288,118.4573,118747.695318,141189.408772,22441.713453
6,SPY,2007-10-30,2007-12-31,1199.377575,117.7189,113.0392,141189.408772,135576.681536,-5612.727236
7,SPY,2008-06-17,2008-07-16,1287.252786,105.3225,96.7927,135576.681536,124596.672723,-10980.008813
8,SPY,2009-05-27,2010-07-06,1742.608370,71.5001,84.0670,124596.672723,146495.857849,21899.185126
9,SPY,2010-10-18,2011-08-03,1507.506440,97.1776,105.1726,146495.857849,158548.371839,12052.513990


In [38]:
#Create plots

price_df = signals_df[['Close', short_name, long_name]]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [39]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest,Buy&Hold]

In [40]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [41]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [42]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)

In [46]:
#For comparison here is a backtest of the 50/150 SMA strategy without any optimization. The optimization has been commented out.


def backtest_no_optimization():
    
    class SmaCross(Strategy):
    
        n1=50
        n2=150
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock2 = Backtest(stock_data_df, SmaCross,
                              cash=100000, commission=.000)
  
#   stats = bt_stock.optimize(n1=range(40, 60, 1),
#                     n2=range(130, 170, 1),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)
   
 
    bt_stock2.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    #return stats
    return bt_stock2.run()
backtest_no_optimization()



Start                     2000-06-30 00:00:00
End                       2020-07-02 00:00:00
Duration                   7307 days 00:00:00
Exposure [%]                          90.9265
Equity Final [$]                       187250
Equity Peak [$]                        324852
Return [%]                            87.2505
Buy & Hold Return [%]                 215.799
Max. Drawdown [%]                    -56.7549
Avg. Drawdown [%]                    -8.71345
Max. Drawdown Duration     1588 days 00:00:00
Avg. Drawdown Duration      121 days 00:00:00
# Trades                                   29
Win Rate [%]                          51.7241
Best Trade [%]                        47.9952
Worst Trade [%]                       -12.467
Avg. Trade [%]                        4.67101
Max. Trade Duration        1113 days 00:00:00
Avg. Trade Duration         230 days 00:00:00
Expectancy [%]                        10.5969
SQN                                   1.15656
Sharpe Ratio                      

In [19]:
# Here we run a backtest on the short = 2, long = 10 strategy, a strategy found when we looked at
# three years of data. We wanted to see how it would fare over twenty years.

def backtest_no_optimization():
    
    class SmaCross(Strategy):
    
        n1=2
        n2=10
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock3 = Backtest(stock_data_df, SmaCross,
                              cash=100000, commission=.000)
  
#   stats = bt_stock.optimize(n1=range(40, 60, 1),
#                     n2=range(130, 170, 1),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)
   
 
    bt_stock3.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    #return stats
    return bt_stock3.run()
backtest_no_optimization()





Start                     2000-07-03 00:00:00
End                       2020-07-02 00:00:00
Duration                   7304 days 00:00:00
Exposure [%]                          99.7125
Equity Final [$]                      31696.8
Equity Peak [$]                        143895
Return [%]                           -68.3032
Buy & Hold Return [%]                  210.15
Max. Drawdown [%]                    -89.0666
Avg. Drawdown [%]                    -29.5103
Max. Drawdown Duration     7203 days 00:00:00
Avg. Drawdown Duration      910 days 00:00:00
# Trades                                  671
Win Rate [%]                          37.1088
Best Trade [%]                        22.4972
Worst Trade [%]                      -8.86033
Avg. Trade [%]                       -0.14325
Max. Trade Duration          74 days 00:00:00
Avg. Trade Duration          11 days 00:00:00
Expectancy [%]                        1.99211
SQN                                  -2.04508
Sharpe Ratio                      